In [1]:
%env QT_QPA_PLATFORM=wayland7
%matplotlib tk
global renderMode
global score
global obs
renderMode = 1

env: QT_QPA_PLATFORM=wayland7


In [2]:
import random
import math
import numpy as np
from matplotlib import pyplot as plt
# import pygame
from time import sleep

In [3]:
size = 8

In [4]:
env_state = 'open'
apple_collected = False

def reset_env():
    global obs
    obs = np.zeros((size, size), dtype=int)
    middle = [math.floor((size+1)/2)-1, math.ceil((size+1)/2)-1]
    y = random.randint(*middle)
    x = random.randint(*middle)
    obs[y][x] = 1 #head
    obs[y+1][x] = 2 #body
    return obs

def generate_apple():
    y = random.randint(0, size-1)
    x = random.randint(0, size-1)
    while obs[y, x] !=0:
        y = random.randint(0, size-1)
        x = random.randint(0, size-1)
    obs[y, x] = -1


def step(action):
    global apple_collected
    apple_collected = False
    tail_value = 0
    head = None
    global obs
    neck = None
    body = None
    tail = None
    for y in range(size):
        for x in range(size):
            if obs[y][x] == 1:
                head = [y, x]
                obs[y][x] += 1
            elif obs[y][x] == 2:
                neck = [y, x]
                obs[y][x] += 1
            elif obs[y][x] > 2:
                body = [y, x]
                obs[y][x] += 1
            if obs[y][x] > tail_value:
                tail = [y, x]
                tail_value = obs[y][x]
    #if tail is None:

    if_apple = obs[tail[0]][tail[1]]
    obs[tail[0]][tail[1]] = 0

    y = head[0]-neck[0]
    x = head[1]-neck[1]
    look_dir = [y, x]

    look_dirs = [[0, -1], [-1, 0], [0, 1], [1, 0]]
    look_dir_index = look_dirs.index(look_dir)
    head_movement = None

    if action == -1: #left
        head_movement = look_dirs[look_dir_index-1]
    elif action == 0: #straight
        head_movement = look_dirs[look_dir_index]
    elif action == 1: #right
        if look_dir_index == 3:
            head_movement = look_dirs[0]
        else:
            head_movement = look_dirs[look_dir_index+1]

    new_head_y = head[0]+head_movement[0]
    new_head_x = head[1]+head_movement[1]

    if (new_head_x > size-1) or (new_head_y > size-1) or (new_head_x < 0) or (new_head_y < 0) or (obs[new_head_y][new_head_x] > 1):
        global env_state
        env_state = 'closed'

    if env_state == 'open':
        if obs[new_head_y][new_head_x] == -1:
            apple_collected = True
            obs[tail[0]][tail[1]] = if_apple
        obs[new_head_y][new_head_x] = 1

    post_step()

def post_step():
    global env_state
    global score
    global obs
    if env_state == 'closed':
        print(score)
        score = 0
        env_state = 'open'
        obs = reset_env()
        for i in range(1):
            generate_apple()

    if apple_collected is True:
        generate_apple()
        score += 1

In [5]:
def init():
    global score
    global obs
    score = 0
    obs = reset_env()
    generate_apple()

In [6]:
if renderMode == 0:
    init()

    for i in range(125):
        action = random.choice([-1, 0, 1])
        step(action)
        #
        # if env_state == 'closed':
        #     print(score)
        #     score = 0
        #     env_state = 'open'
        #     obs = reset_env()
        #     for i in range(1):
        #         generate_apple()
        #
        # if apple_collected is True:
        #     generate_apple()
        #     score += 1

        plt.ion()
        plt.imshow(obs)
        plt.pause(0.01)
        sleep(0.1)

In [7]:
# obs = reset_env()
# generate_apple()

In [8]:
# step(0)

In [9]:
# plt.ion()
# plt.imshow(obs)
# #plt.pause(0.5)

In [10]:

# def print_obs(): #doesnt work and to be replaced by pygame plug
#     obs2print = obs
#     for y in range(size):
#         for x in range(size):
#             if obs2print[y, x] == 0:
#                 obs2print[y, x] = [0, 0, 0]
#             elif obs2print[y, x] > 0:
#                 obs2print[y, x] = [0, 0, 255]
#             elif obs2print[y, x] == -1:
#                 obs2print[y, x] = [255, 0, 0]
#             else:
#                 obs2print[y, x] = [255, 255, 255]
#     %matplotlib inline
#     plt.imshow(obs2print)

In [ ]:
import enum

class logTypes(enum.Enum):
    Info = 0
    Ok = 1
    Warn = 2
    Error = 3

class inputModes(enum.Enum):
    PlayerNative = 0
    PlayerTrans = 1
    PreComputed = 2
    Hook = 3
    PlayerAuto = 4

if renderMode == 1:
    global obs, logList, maxVal
    import pygame

    pygame.init()

    # input
    im = inputModes.PlayerAuto

    # config
    s    = 80              # cell size in px
    b    = 0               # border size in px
    rl   = True            # render log or not
    rg   = True            # render grid or not
    rs   = True            # render score or not
    fs   = 26              # font size
    fps  = 60              # target frames per second
    clrV = (0, 0, 0)       # void (background)
    clrA = (255, 0, 0)     # apple
    clrH = (0, 255, 0)     # head
    clrB = (0, 255, 0)     # body
    clrF = (255, 255, 255) # foreground
    clrG = (40, 40, 40)    # grid

    c = 8
    w = c * s
    h = c * s
    oq = s / 4
    oh = s / 2

    if rl:
        w += s * 8

    ds = pygame.display.set_mode((w, h))
    clock = pygame.time.Clock()
    logList = []
    font = pygame.font.Font(pygame.font.get_default_font(), fs)
    maxVal = 0

    def getMaxVal():
        m = 0
        for y in obs:
            for x in y:
                if x > m:
                    m = x
        return m

    def getValIndex(val):
        index = 0
        for y in obs:
            for x in y:
                if x == val:
                    return index
                index += 1
        return -1

    def getValPos(val):
        index = getValIndex(val)
        log(index)
        if index == -1:
            return [-1, -1]
        yP = index // c
        xP = index - yP * c
        log("xp: " + str(xP) + " yp:" + str(yP))
        return [xP, yP]

    def orient(xP, yP, val):
        pp = getValPos(val - 1)
        pn = getValPos(val + 1)
        isMax = val == maxVal
        log("jsem blok: " + str(val) + " pp: " + str(pp) + " pn:" + str(pn) + " jsem posledni: " + str(isMax))
        if pp[0] == -1:
            return False
        if pp[0] - 1 == xP and pp[1] == yP:
            if isMax:
                drawTriangle(clrB, (xP + 1) * s, yP * s + oq, (xP + 1) * s, yP * s + oh + oq, xP * s, yP * s + oh)
            else:
                if pn[0] != -1:
                    if pn[1] == yP + 1:
                        drawRect(clrB, xP * s + oq, yP * s + oq, s - oq, oh)
                        drawRect(clrB, xP * s + oq, yP * s + oq + oh, oh, oq)
                        return True
                    elif pn[1] == yP - 1:
                        drawRect(clrB, xP * s + oq, yP * s + oq, s - oq, oh)
                        drawRect(clrB, xP * s + oq, yP * s, oh, oq)
                        return True
                drawRect(clrB, xP * s, yP * s + oq, s, oh)
            return True
        if pp[0] + 1 == xP and pp[1] == yP:
            if isMax:
                drawTriangle(clrB, xP * s, yP * s + oq, xP * s, yP * s + oh + oq, (xP + 1) * s, yP * s + oh)
            else:
                if pn[0] != -1:
                    if pn[1] == yP + 1:
                        drawRect(clrB, xP * s, yP * s + oq, s - oq, oh)
                        drawRect(clrB, xP * s + oq, yP * s + oq + oh, oh, oq)
                        return True
                    elif pn[1] == yP - 1:
                        drawRect(clrB, xP * s, yP * s + oq, s - oq, oh)
                        drawRect(clrB, xP * s + oq, yP * s, oh, oq)
                        return True
                drawRect(clrB, xP * s, yP * s + oq, s, oh)
            return True
        if pp[0]== xP and pp[1] - 1 == yP:
            if isMax:
                drawTriangle(clrB, xP * s + oq, (yP + 1) * s, xP * s + oh + oq, (yP + 1) * s, xP * s + oh, yP * s)
            else:
                if pn[0] != -1:
                    if pn[0] == xP - 1:
                        drawRect(clrB, xP * s + oq, yP * s + oq, oh, s - oq)
                        drawRect(clrB, xP * s, yP * s + oq, oq, oh)
                        return True
                    elif pn[0] == xP + 1:
                        drawRect(clrB, xP * s + oq, yP * s + oq, oh, s - oq)
                        drawRect(clrB, xP * s + oh + oq, yP * s + oq, oq, oh)
                        return True
                drawRect(clrB, xP * s + oq, yP * s, oh, s)
            return True
        if pp[0]== xP and pp[1] + 1 == yP:
            if isMax:
                drawTriangle(clrB, xP * s + oq, yP * s, xP * s + oh + oq, yP * s, xP * s + oh, (yP + 1) * s)
            else:
                if pn[0] != -1:
                    if pn[0] == xP - 1:
                        drawRect(clrB, xP * s + oq, yP * s, oh, s - oq)
                        drawRect(clrB, xP * s, yP * s + oq, oq, oh)
                        return True
                    elif pn[0] == xP + 1:
                        drawRect(clrB, xP * s + oq, yP * s, oh, s - oq)
                        drawRect(clrB, xP * s + oh + oq, yP * s + oq, oq, oh)
                        return True
                drawRect(clrB, xP * s + oq, yP * s, oh, s)
            return True
        return False

    def cell(xP, yP, val):
        if val == 0:
            return
        else:
            clr = clrA
            if val == -1:
                drawCircle(clr, xP * s + oh, yP * s + oh, s)
                return
            if val == 1:
                pn = getValPos(val + 1)
                if pn[0] != -1:
                    ppp = 0
                    # if pn[0] == xP and pn[1] == yP - 1:
                    #     drawPoly(clrA, [
                    #         (xP * s + oq, (yP + 1) * s),
                    #         (xP * s, yP * s + oh),
                    #         (xP * s + oq, yP * s),
                    #         (yP * s + oq + oh, yP * s),
                    #         ((xP + 1) * s, yP * s + oh),
                    #         (xP * s + oq + oh, (yP + 1) * s),
                    #         (xP * s + oq, (yP + 1) * s)
                    #     ])
                    #     return
                clr = clrH
            elif val >= 2:
                clr = clrB
                t = orient(xP, yP, val)
                if t:
                    return
            drawRect(clr, xP * s, yP * s, s, s)

    def log(text, type = logTypes.Info):
        global logList
        logList.append([str(text), type])

    def drawLine(c, x1, y1, x2, y2, w = 1):
        pygame.draw.line(ds, c, (x1, y1), (x2, y2), w)

    def drawPoly(c, vert):
        pygame.draw.polygon(ds, c, vert)

    def drawQuad(c, x1, y1, x2, y2, x3, y3, x4, y4):
        pygame.draw.polygon(ds, c, [(x1, y1), (x2, y2), (x3, y3), (x4, y4)])

    def drawTriangle(c, x1, y1, x2, y2, x3, y3):
        pygame.draw.polygon(ds, c, [(x1, y1), (x2, y2), (x3, y3)])

    def drawRect(c, x, y, w, h):
        pygame.draw.rect(ds, c, pygame.Rect(x, y,  w,  h))

    def drawRectExt(c, x, y, w, h, ow):
        pygame.draw.rect(ds, c, pygame.Rect(x, y,  w,  h), ow)

    def drawText(c, x, y, t):
       ts = font.render(t, True, c)
       ds.blit(ts, (x, y))

    def drawCircle(c, x, y, r):
        pygame.draw.circle(ds, c, (x, y), r / 2)

    def renderLogList():
        if not rl:
            return
        global logList
        index = 0
        for entry in logList:
            c = clrF
            if entry[1] == logTypes.Ok:
                c = (0, 255, 0)
            elif entry[1] == logTypes.Warn:
                c = (255, 165, 0)
            elif entry[1] == logTypes.Error:
                c = (255, 0, 0)
            drawText(c, w / 2 + 10, 10 + index * (fs + 4), entry[0])
            index += 1
        logList = []

    def beforeRender():
        global maxVal
        maxVal = getMaxVal()

    def clear():
        drawRect(clrV, 0, 0, w, h)

    def renderGrid():
        index = 0
        for _ in obs:
            drawLine(clrG, index * s, 0, index * s, s * c)
            drawLine(clrG, 0, index * s, s * c, index * s)
            index += 1

    def renderScore():
        drawText(clrF, 6, 6, "Skore: " + str(score))

    def input():
        return

    def render():
        beforeRender()
        clear()

        if rg:
            renderGrid()

        if rl:
            drawRectExt(clrF, w / 2, 0, w / 2, h, 1)

        xx = 0
        yy = 0
        for y in obs:
            for x in y:
                cell(xx, yy, x)
                xx += 1
            yy += 1
            xx = 0
        renderLogList()

        if rs:
            renderScore()

    run = True
    init()
    while run:
        clock.tick(fps)
        for e in pygame.event.get():
            if e.type == pygame.QUIT:
                pygame.quit()
                run = False
            elif e.type == pygame.KEYDOWN:
                if e.key == pygame.K_w:
                    step(0)
                elif e.key == pygame.K_d:
                    step(1)
                elif e.key == pygame.K_a:
                    step(-1)
        render()
        pygame.display.update()


pygame 2.1.2 (SDL 2.0.16, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
30
4
5
5
8
5
35
11


# vstup
# vykreslovani
# herni logika